### Setup + Load

In [1]:
# imports
from pathlib import Path
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator, array_to_img
import albumentations as A
from tensorflow.keras.utils import Sequence
from tqdm import tqdm  # for progress bar
import cv2

In [2]:
# # prevent my poor mac from overheating
# os.environ["OMP_NUM_THREADS"] = "4" 

In [3]:
# load image metadata
DATA_ROOT = Path("/Volumes/Crucial X10/210/data/columbia_gaze_dataset")

rows = []

# Regex for filename parsing
pattern = re.compile(
    r"(?P<subject>\d+)_"
    r"(?P<distance>\d+)m_"
    r"(?P<head_pose>-?\d+)P_"
    r"(?P<gaze_v>-?\d+)V_"
    r"(?P<gaze_h>-?\d+)H\.jpg"
)

for subject_dir in DATA_ROOT.iterdir():
    if not subject_dir.is_dir():
        continue

    for img_path in subject_dir.glob("*.jpg"):
        match = pattern.match(img_path.name)
        if not match:
            continue  # skip unexpected filenames

        meta = match.groupdict()

        rows.append({
            "path": str(img_path),
            "filename": img_path.name,
            "subject": meta["subject"],
            "distance_m": int(meta["distance"]),
            "head_pose_deg": int(meta["head_pose"]),
            "gaze_vertical_deg": int(meta["gaze_v"]),
            "gaze_horizontal_deg": int(meta["gaze_h"]),
        })

df = pd.DataFrame(rows)

In [4]:
df.head()

,path,filename,subject,distance_m,head_pose_deg,gaze_vertical_deg,gaze_horizontal_deg
0,/Volumes/Crucial X10/210/data/columbia_gaze_da...,0050_2m_-30P_0V_-10H.jpg,0050,2,-30,0,-10
1,/Volumes/Crucial X10/210/data/columbia_gaze_da...,0050_2m_30P_-10V_5H.jpg,0050,2,30,-10,5
2,/Volumes/Crucial X10/210/data/columbia_gaze_da...,0050_2m_30P_-10V_-5H.jpg,0050,2,30,-10,-5
3,/Volumes/Crucial X10/210/data/columbia_gaze_da...,0050_2m_-15P_10V_-10H.jpg,0050,2,-15,10,-10
4,/Volumes/Crucial X10/210/data/columbia_gaze_da...,0050_2m_-15P_0V_5H.jpg,0050,2,-15,0,5


### Create Labels

In [5]:
# create labels based on degrees
def create_labels(row):
    '''
    converts per-image gaze metadata into an intent-level
    classification
    '''
    h = row["gaze_horizontal_deg"]
    v = row["gaze_vertical_deg"]

    # straight
    if v==0 and h==0: 
        return "straight"
    
    # horizontal dominates
    if abs(h) > abs(v):
        return "left" if h < 0 else "right"

    # vertical dominates
    if abs(v) > abs(h):
        return "down" if v < 0 else "up"

    # tie → horizontal wins (gaze is steadier in horizontal axis)
    return "left" if h < 0 else "right"

df["label"] = df.apply(create_labels, axis=1)

### Augmentation Pipeline

In [6]:
# Paths
DATA_ROOT = Path("/Volumes/Crucial X10/210/data/columbia_gaze_dataset")
OUTPUT_ROOT = Path("/Volumes/Crucial X10/210/data/columbia_gaze_augmented")

# will end up with dataset size = N original images x augmentations
NUM_AUGMENTATIONS = 5 

# distance varies wildly in the wild so create diff size pics
TARGET_SIZES = [(224, 224), (192, 192), (256, 256)]

# num of images to process at once; adjust based on needs
# 75 is recommended for M1 mac with adequate disk space
BATCH_SIZE = 75  

In [11]:
augmentation = A.Compose([
    # Geometric transformations - simulate different distances/angles
    A.ShiftScaleRotate(
        shift_limit=0.15,      # More head position variance
        scale_limit=0.25,      # Simulate 1m to 3m+ distances
        rotate_limit=20,       # More head tilt tolerance
        border_mode=cv2.BORDER_CONSTANT,
        p=0.8
    ),
    
    # Perspective changes - simulate camera angles
    A.Perspective(
        scale=(0.05, 0.15),
        p=0.5
    ),
    
    # Lighting variations - crucial for real-world
    A.OneOf([
        A.RandomBrightnessContrast(
            brightness_limit=0.3,
            contrast_limit=0.3,
            p=1.0
        ),
        A.RandomGamma(gamma_limit=(70, 130), p=1.0),
        A.CLAHE(clip_limit=4.0, p=1.0),  # Handles harsh lighting
    ], p=0.8),
    
    # Color temperature shifts
    A.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1,
        p=0.5
    ),
    
    # Simulate poor lighting conditions
    A.RandomShadow(
        shadow_roi=(0, 0, 1, 1),
        num_shadows_limit=(1,2),
        p=0.3
    ),
    
    # Camera quality degradation
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=1.0),
        A.MotionBlur(blur_limit=7, p=1.0),  # Movement/tremors
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
    ], p=0.5),
    
    # Compression artifacts (like video calls)
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.4),
    
    # Occasional severe conditions
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1.0),
        A.RandomRain(p=1.0),  # Window reflections, etc.
    ], p=0.1),
])

/var/folders/89/572hqr1x26j5sf69pfl_2fd80000gn/T/ipykernel_21431/3840305309.py:48: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
/var/folders/89/572hqr1x26j5sf69pfl_2fd80000gn/T/ipykernel_21431/3840305309.py:52: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=60, quality_upper=100, p=0.4),
/var/folders/89/572hqr1x26j5sf69pfl_2fd80000gn/T/ipykernel_21431/3840305309.py:56: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1.0),


In [16]:
def augment_image(img, label):
    # Random resize to simulate different distances
    target_size = TARGET_SIZES[np.random.randint(0, len(TARGET_SIZES))]  # ← Fixed
    img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    
    transformed = augmentation(image=img_resized)
    aug_img = transformed['image']
    
    # Final resize to consistent size for training
    aug_img = cv2.resize(aug_img, (224, 224), interpolation=cv2.INTER_AREA)
    
    return aug_img, label

In [13]:
def process_batch(batch_df, output_root, num_augmentations):
    """Process a single batch of images."""
    results = []
    
    for _, row in batch_df.iterrows():
        # Load and resize image
        img = cv2.imread(row['path'])
        if img is None:
            print(f"Warning: Could not load {row['path']}")
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Create output directory for this subject
        subject_dir = output_root / row['subject']
        subject_dir.mkdir(parents=True, exist_ok=True)
        
        # Save resized original
        stem = Path(row['filename']).stem
        
        # Process original (pick random size for variety)
        target_size = TARGET_SIZES[np.random.randint(0, len(TARGET_SIZES))]
        img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
        img_final = cv2.resize(img_resized, (224, 224), interpolation=cv2.INTER_AREA)
        
        orig_path = subject_dir / f"{stem}.jpg"
        cv2.imwrite(str(orig_path), cv2.cvtColor(img_final, cv2.COLOR_RGB2BGR))
        
        # Add original to results
        results.append({
            **row.to_dict(),
            'augmented_path': str(orig_path),
            'is_augmented': False,
            'aug_id': 0
        })
        
        # Generate augmentations
        for aug_idx in range(1, num_augmentations + 1):
            aug_img, aug_label = augment_image(img, row['label'])
            
            aug_path = subject_dir / f"{stem}_aug{aug_idx}.jpg"
            cv2.imwrite(str(aug_path), cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))
            
            results.append({
                **row.to_dict(),
                'augmented_path': str(aug_path),
                'label': aug_label,
                'is_augmented': True,
                'aug_id': aug_idx
            })
        
        del img
    
    return results

In [14]:
def process_full_dataset(df, output_root, batch_size, num_augmentations):
    """Process entire dataset in batches."""
    import gc
    
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)
    
    total = len(df)
    num_batches = (total + batch_size - 1) // batch_size
    
    print(f"\nProcessing {total} images in {num_batches} batches of {batch_size}")
    print(f"Creating {num_augmentations} augmentations per image")
    print(f"Total output: {total * (num_augmentations + 1)} images\n")
    
    all_results = []
    
    for batch_idx in tqdm(range(num_batches), desc="Augmenting"):
        start = batch_idx * batch_size
        end = min(start + batch_size, total)
        batch_df = df.iloc[start:end]
        
        batch_results = process_batch(batch_df, output_root, num_augmentations)
        all_results.extend(batch_results)
        
        # Force cleanup
        gc.collect()
    
    return pd.DataFrame(all_results)

### Test Pipeline

In [17]:
# Create a test batch (first 75 images)
test_batch = df.iloc[:75]

# Create a test output directory
test_output = Path("/Volumes/Crucial X10/210/data/test_augmentation")
test_output.mkdir(parents=True, exist_ok=True)

# Run the single batch
print("🧪 Testing with one batch (75 images)...")
test_results = process_batch(
    batch_df=test_batch,
    output_root=test_output,
    num_augmentations=NUM_AUGMENTATIONS
)

# Convert to DataFrame to see results
test_df = pd.DataFrame(test_results)

print(f"\n✅ Test complete!")
print(f"Processed {len(test_batch)} original images")
print(f"Created {len(test_df)} total images (originals + augmentations)")
print(f"\nLabel distribution:\n{test_df['label'].value_counts()}")
print(f"\nFiles saved to: {test_output}")

# Show a few examples
print(f"\nFirst few augmented files:")
print(test_df[['filename', 'label', 'is_augmented', 'aug_id']].head(10))

🧪 Testing with one batch (75 images)...

✅ Test complete!
Processed 75 original images
Created 450 total images (originals + augmentations)

Label distribution:
label
left        156
right       138
down         78
up           54
straight     24
Name: count, dtype: int64

Files saved to: /Volumes/Crucial X10/210/data/test_augmentation

First few augmented files:
                   filename label  is_augmented  aug_id
0  0050_2m_-30P_0V_-10H.jpg  left         False       0
1  0050_2m_-30P_0V_-10H.jpg  left          True       1
2  0050_2m_-30P_0V_-10H.jpg  left          True       2
3  0050_2m_-30P_0V_-10H.jpg  left          True       3
4  0050_2m_-30P_0V_-10H.jpg  left          True       4
5  0050_2m_-30P_0V_-10H.jpg  left          True       5
6   0050_2m_30P_-10V_5H.jpg  down         False       0
7   0050_2m_30P_-10V_5H.jpg  down          True       1
8   0050_2m_30P_-10V_5H.jpg  down          True       2
9   0050_2m_30P_-10V_5H.jpg  down          True       3


### Run Pipeline

In [18]:
# Run augmentation
print("🚀 Starting augmentation process...")
augmented_df = process_full_dataset(
    df=df,
    output_root=OUTPUT_ROOT,
    batch_size=BATCH_SIZE,
    num_augmentations=NUM_AUGMENTATIONS
)

print(f"\n✅ Augmentation complete!")
print(f"Total images: {len(augmented_df)}")
print(f"Label distribution:\n{augmented_df['label'].value_counts()}")

# Save metadata
metadata_path = OUTPUT_ROOT / "augmented_metadata.csv"
augmented_df.to_csv(metadata_path, index=False)
print(f"\n💾 Saved metadata to: {metadata_path}")

🚀 Starting augmentation process...

Processing 5880 images in 79 batches of 75
Creating 5 augmentations per image
Total output: 35280 images



Augmenting: 100%|██████████| 79/79 [45:52<00:00, 34.84s/it]


✅ Augmentation complete!
Total images: 35280
Label distribution:
label
left        11760
right       11760
down         5040
up           5040
straight     1680
Name: count, dtype: int64

💾 Saved metadata to: /Volumes/Crucial X10/210/data/columbia_gaze_augmented/augmented_metadata.csv
